# EDA Analysis and model experimentation #

In [1]:
# Importing libraries
import os
import pandas as pd
import numpy as np

from src import config
from src.data_utils import get_dataframes
import matplotlib.pyplot as plt
import seaborn as sns

from src.plots import plot_missing_vals

import openai
import pandas as pd
import boto3
from model import model
import langchain

# Importing ENV Variables

In [2]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
OPEN_AI_API_KEY = os.getenv('OPEN_AI_API_KEY') # API key for OpenAI
AWS_S3= os.getenv('AWS_S3')
AWS_S3_SECRET = os.getenv('AWS_S3_SECRET')

## Instanciating a connection to S3 with credentials

In [4]:

s3 = boto3.client('s3', 
                  aws_access_key_id=AWS_S3, 
                  aws_secret_access_key=AWS_S3_SECRET
                  )


## Getting the files filter by prefix

In [5]:

from src import config
from pathlib import Path
prefix = 'queplan_insurance/'
bucket_name = 'anyoneai-datasets'   
maxkeys = 9999999
s3 = boto3.client('s3')
list=s3.list_objects(Bucket=bucket_name, Prefix=prefix, MaxKeys = maxkeys)['Contents']
list_keys = []
list_filepath = []
dataset_path = str(Path(config.DATASET_ROOT_PATH) / "raw_pdfs")

for s3_key in list:
    s3_object = s3_key['Key']
    
    path, filename = os.path.split(s3_object)
    
    filepath = os.path.join(dataset_path, filename)
    list_filepath.append(filepath) 
    list_keys.append(s3_object)

In [6]:
list_filepath

['/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL120190177.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320130223.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320180100.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320200071.pdf',
 '/home/andy/In

In [7]:
list_keys

['queplan_insurance/',
 'queplan_insurance/POL120190177.pdf',
 'queplan_insurance/POL320130223.pdf',
 'queplan_insurance/POL320150503.pdf',
 'queplan_insurance/POL320180100.pdf',
 'queplan_insurance/POL320190074.pdf',
 'queplan_insurance/POL320200071.pdf',
 'queplan_insurance/POL320200214.pdf',
 'queplan_insurance/POL320210063.pdf',
 'queplan_insurance/POL320210210.pdf']

## Downloading the specific files by prefix 'queplan_insurance'

In [8]:
s3 = boto3.resource('s3')

for key,path in zip(list_keys[1:],list_filepath[1:]):
    if os.path.exists(path):
        print(f"File already downloaded: {path}")
    else:
        s3.meta.client.download_file(bucket_name, key,path)

File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL120190177.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320130223.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320180100.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320200071.pdf
File already dow

In [9]:
# [TO DO] - Add code to download the data from S3 bucket as a function in data utils
# def download_data_from_s3():

In [10]:
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader
pdf_list = os.listdir(str(Path(config.DATASET_ROOT_PATH) /  "raw_pdfs"))

list = []
pdf_list


['POL320150503.pdf',
 'POL320190074.pdf',
 'POL320210063.pdf',
 'POL320180100.pdf',
 'POL320210210.pdf',
 'POL320130223.pdf',
 'POL320200214.pdf',
 'POL320200071.pdf',
 'POL120190177.pdf']

In [11]:
config.DATASET_ROOT_PATH

'/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset'

In [12]:
for pdf in pdf_list[1:]:
    loader_pymu = PyMuPDFLoader(f"{config.DATASET_ROOT_PATH}/raw_pdfs/{pdf}")  # Load the document
    pages_pymu = loader_pymu.load()
    list.append(pages_pymu)

In [13]:
list[5][0]

Document(page_content='SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO\nIncorporada al Depósito de Pólizas bajo el código POL320200214\n \nARTICULO 1°: REGLAS APLICABLES AL CONTRATO\n \n \nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las\nnormas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin\nembargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el\nasegurado o dependiente.\n \n \nLa presente póliza se otorga en base a las declaraciones, informaciones y antecedentes proporcionados por\nel asegurado titular a solicitud del Asegurador, y en base a la información que ha entregado el Asegurador al\nasegurado titular respecto a las condiciones, términos y modalidades del seguro, todos los cuales forman\nparte integrante de la presente póliza.\n \n \nARTÍCULO 2º: COBERTURA\n \n \nEl Asegurador reembolsará los gastos médicos razonables, ac

In [14]:
pdf_list

['POL320150503.pdf',
 'POL320190074.pdf',
 'POL320210063.pdf',
 'POL320180100.pdf',
 'POL320210210.pdf',
 'POL320130223.pdf',
 'POL320200214.pdf',
 'POL320200071.pdf',
 'POL120190177.pdf']

In [15]:
from langchain.document_loaders import DirectoryLoader
loader_dl = DirectoryLoader("dataset/raw_pdfs/")
pages_dl = loader_dl.load()

In [16]:
pages_dl[5].metadata["source"]

'dataset/raw_pdfs/POL320130223.pdf'

In [17]:
import tiktoken
from langchain.document_loaders import DirectoryLoader
loader_dl = DirectoryLoader("dataset/raw_pdfs/")
pages_dl = loader_dl.load()

MODEL = "gpt-3.5-turbo"

encoder = tiktoken.encoding_for_model(MODEL)


tokens_dict = {}
for page in pages_dl:
    num_tokens = len(encoder.encode(page.page_content))
    tokens_dict[page.metadata["source"]] = num_tokens

In [18]:
tokens_dict

{'dataset/raw_pdfs/POL320150503.pdf': 16949,
 'dataset/raw_pdfs/POL320190074.pdf': 28058,
 'dataset/raw_pdfs/POL320210063.pdf': 4192,
 'dataset/raw_pdfs/POL320180100.pdf': 15782,
 'dataset/raw_pdfs/POL320210210.pdf': 6838,
 'dataset/raw_pdfs/POL320130223.pdf': 13783,
 'dataset/raw_pdfs/POL320200214.pdf': 27308,
 'dataset/raw_pdfs/POL320200071.pdf': 23280,
 'dataset/raw_pdfs/POL120190177.pdf': 16037}

In [19]:
pages_dl[1].page_content

'SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO\n\nIncorporada al Depósito de Pólizas bajo el código POL320190074\n\nARTICULO 1°: REGLAS APLICABLES AL CONTRATO\n\nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o beneficiario.\n\nARTÍCULO 2º: COBERTURA\n\nLa compañía aseguradora reembolsará los gastos médicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un Evento cubierto por esta póliza, en los términos y condiciones establecidas en ésta, siempre que haya transcurrido el periodo de carencia establecido en las Condiciones Particulares, que la póliza se encuentre vigente y que no haya transcurrido el plazo definido en las Condiciones Particulares para la cobertura del Evento.

In [20]:
import importlib
import src.text_preprocessing as tp

[nltk_data] Downloading package stopwords to /home/andy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/andy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


    corpus: List[str],
    html_stripping: Optional[bool] = True,
    accented_char_removal: Optional[bool] = True,
    text_lower_case: Optional[bool] = True,
    text_stemming: Optional[bool] = False,
    text_lemmatization: Optional[bool] = False,
    special_char_removal: Optional[bool] = True,
    remove_digits: Optional[bool] = True,
    stopword_removal: Optional[bool] = True,
    stopwords: Optional[List[str]] = stopword_list,

In [21]:
text = pages_dl[1].page_content

In [22]:
text_wo_char = tp.normalize_corpus([text], special_char_removal=True, stopword_removal=False, remove_digits=False)
text_wo_char , len(text_wo_char[0])

(['seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo pol320190074\n\narticulo 1: reglas aplicables al contrato\n\nse aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii, del libro ii, del codigo de comercio. sin embargo, se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o beneficiario.\n\narticulo 2o: cobertura\n\nla compania aseguradora reembolsara los gastos medicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un evento cubierto por esta poliza, en los terminos y condiciones establecidas en esta, siempre que haya transcurrido el periodo de carencia establecido en las condiciones particulares, que la poliza se encuentre vigente y que no haya transcurrido el plazo definido en las condiciones particulares para la cobertura del evento

In [23]:
text_w_char = tp.normalize_corpus([text], special_char_removal=False, stopword_removal=False,remove_digits=False)
text_w_char , len(text_w_char[0])

(['seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo pol320190074\n\narticulo 1°: reglas aplicables al contrato\n\nse aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii, del libro ii, del codigo de comercio. sin embargo, se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o beneficiario.\n\narticulo 2o: cobertura\n\nla compania aseguradora reembolsara los gastos medicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un evento cubierto por esta poliza, en los terminos y condiciones establecidas en esta, siempre que haya transcurrido el periodo de carencia establecido en las condiciones particulares, que la poliza se encuentre vigente y que no haya transcurrido el plazo definido en las condiciones particulares para la cobertura del event

In [24]:
import re
text_w_char = tp.normalize_corpus([text], special_char_removal=False, stopword_removal=False,remove_digits=False)
pattern = r"(\narticulo .*.*\s*\d+\d*).*:{1}\s*(\s*[^\n]*)((?=[\s*]*))"
articles = re.findall(pattern, text_w_char[0])

In [25]:
print(articles)

[('\narticulo 1', 'reglas aplicables al contrato', ''), ('\narticulo 2', 'cobertura', ''), ('\narticulo 3', 'definiciones.', ''), ('\narticulo 4', 'exclusiones.', ''), ('\narticulo 5', 'carencia', ''), ('\narticulo 6', 'obligaciones del asegurado', ''), ('\narticulo 7', 'declaraciones del asegurado', ''), ('\narticulo 8', 'primas y efectos del no pago de las primas', ''), ('\narticulo 9', 'beneficiarios', ''), ('\narticulo 10', 'denuncia de siniestros', ''), ('\narticulo 11', 'vigencia y terminacion', ''), ('\narticulo 12', 'comunicacion entre las partes', ''), ('\narticulo 13', 'solucion de controversias', ''), ('\narticulo 14', 'clausulas adicionales', ''), ('\narticulo 15', 'domicilio', ''), ('\narticulo n° 1', 'cobertura.', ''), ('\narticulo n° 2', 'exclusiones.', ''), ('\narticulo n° 3', 'carencia', ''), ('\narticulo no 4', 'beneficiarios', ''), ('\narticulo n° 5', 'denuncia de siniestro', ''), ('\narticulo 6', 'clausulas aplicables', ''), ('\narticulo 1', 'reglas aplicables al co

In [26]:
pattern_policies = r"([a-z]{3}\d+)"
match_dict = {}
for doc in text_w_char:
    matches = re.split(pattern_policies, text_w_char[0])

In [27]:
matches

['seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo ',
 'pol320190074',
 '\n\narticulo 1°: reglas aplicables al contrato\n\nse aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii, del libro ii, del codigo de comercio. sin embargo, se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o beneficiario.\n\narticulo 2o: cobertura\n\nla compania aseguradora reembolsara los gastos medicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un evento cubierto por esta poliza, en los terminos y condiciones establecidas en esta, siempre que haya transcurrido el periodo de carencia establecido en las condiciones particulares, que la poliza se encuentre vigente y que no haya transcurrido el plazo definido en las condiciones particulares para la cobertura 

In [28]:
import PyPDF2

policies = {}
policies_list = []
for pdf in pdf_list:
    pattern = r"([A-za-z]{3}\d{3,15})"
    filepath = str(Path(config.DATASET_ROOT_PATH) /  f"raw_pdfs/{pdf}")
    with open(filepath, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pattern_found = False
        start_page = None
        

        policies_metadata = {"policy_number": None, "start_page": None, "end_page": None, "filepath": None, "num_pages": None}
        num_pages = len(pdf_reader.pages)
        policies_num = 0
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            pattern_found = re.findall(pattern, page_text)
            
            
            if pattern_found:
                if policies_num == 0:
                    policies_num += 1
                    policies_metadata = {"policy_number": pattern_found[0], "start_page": page_num, "end_page": None, "filepath": filepath, "num_pages": None}
                    policies_list.append(policies_metadata)
                elif policies_num > 0:
                    policies_num += 1
                    end_page = page_num
                    policies_list[-1]["end_page"] = end_page-1
                    start_page = policies_list[-1]["start_page"]
                    num_pages = end_page - start_page
                    policies_list[-1]["num_pages"] = num_pages
                    policies_metadata = {"policy_number": pattern_found[0], "start_page": page_num, "end_page": None, "filepath": filepath, "num_pages": None}
                    policies_list.append(policies_metadata)
    policies_list[-1]["num_pages"] = num_pages
    policies_list[-1]["end_page"] = page_num
            

In [29]:
policies_list

[{'policy_number': 'POL320150503',
  'start_page': 0,
  'end_page': 25,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf',
  'num_pages': 26},
 {'policy_number': 'POL320190074',
  'start_page': 0,
  'end_page': 9,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
  'num_pages': 10},
 {'policy_number': 'CAD220130244',
  'start_page': 10,
  'end_page': 14,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
  'num_pages': 5},
 {'policy_number': 'POL320130223',
  'start_page': 15,
  'end_page': 61,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
  'num_pages': 47},
 {'polic

In [30]:
for doc in policies_list:
    pn, sp, ep, fp, np = doc.values()
    with open(fp, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        output_pdf = PyPDF2.PdfWriter()
        output_file = f'{config.DATASET_ROOT_PATH}/raw_chunks/{pn}.pdf'
        if not os.path.exists(output_file):
            for i in range(sp, ep+1):
                output_pdf.add_page(pdf_reader.pages[i])
            with open(f'{config.DATASET_ROOT_PATH}/raw_chunks/{pn}.pdf', 'wb') as output_file:
                    output_pdf.write(output_file)
        else:
            print(f"""File {pn} already exists.
                  check manually for inconsistencies in file {fp} with the following original raw doc""")

File POL320150503 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf with the following original raw doc
File POL320190074 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf with the following original raw doc
File CAD220130244 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf with the following original raw doc
File POL320130223 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_ex

In [31]:
import tiktoken
from langchain.document_loaders import DirectoryLoader
loader_dl = DirectoryLoader("dataset/raw_chunks/")
pages_dl = loader_dl.load()

EMBEDDINGS_MODEL = "text-embedding-ada-002"
MODEL = "gpt-3.5-turbo"
MODEL_16K = "gpt-3.5-turbo-16k"

encoder = tiktoken.encoding_for_model(EMBEDDINGS_MODEL)

In [32]:



tokens_dict = {}
for doc in pages_dl:
    content = doc.page_content
    num_tokens = len(encoder.encode(content))
    text_w_char = tp.normalize_corpus([content], special_char_removal=False, stopword_removal=False,remove_digits=False)
    pattern = r"(\narticulo .*.*\s*\d+\d*).*:{0,1}\.*\s*(\s*[^\n]*)((?=[\s*]*))"
    articles = len(re.findall(pattern, text_w_char[0]))
    tokens_dict[doc.metadata["source"]] = {"num_tokens": num_tokens , "num_articles": articles}
tokens_dict

{'dataset/raw_chunks/CAD220130244.pdf': {'num_tokens': 1912,
  'num_articles': 6},
 'dataset/raw_chunks/CAD320190121.pdf': {'num_tokens': 3656,
  'num_articles': 6},
 'dataset/raw_chunks/POL320160108.pdf': {'num_tokens': 7987,
  'num_articles': 20},
 'dataset/raw_chunks/POL320150503.pdf': {'num_tokens': 16949,
  'num_articles': 23},
 'dataset/raw_chunks/POL320190074.pdf': {'num_tokens': 6806,
  'num_articles': 15},
 'dataset/raw_chunks/CAD220130227.pdf': {'num_tokens': 1901,
  'num_articles': 6},
 'dataset/raw_chunks/POL320210063.pdf': {'num_tokens': 4192,
  'num_articles': 15},
 'dataset/raw_chunks/POL320180100.pdf': {'num_tokens': 15782,
  'num_articles': 23},
 'dataset/raw_chunks/POL320210210.pdf': {'num_tokens': 6838,
  'num_articles': 15},
 'dataset/raw_chunks/POL320130223.pdf': {'num_tokens': 13783,
  'num_articles': 19},
 'dataset/raw_chunks/POL320200214.pdf': {'num_tokens': 27308,
  'num_articles': 20},
 'dataset/raw_chunks/POL320200071.pdf': {'num_tokens': 15293,
  'num_articl

In [33]:
text_dict = {}
pattern = r"(\narticulo .*.*\s*\d+\d*).*:{0,1}\.*\s*(\s*[^\n]*)((?=[\s*]*))"

for doc in pages_dl:
    content = doc.page_content
    text_w_char = tp.normalize_corpus([content], special_char_removal=False, stopword_removal=False,remove_digits=False)
    num_tokens = len(encoder.encode(content))
    articles = len(re.findall(pattern, text_w_char[0]))
    text_dict[doc.metadata["source"]] = {"num_tokens": num_tokens , "num_articles": articles, "text": text_w_char}

text_dict


{'dataset/raw_chunks/CAD220130244.pdf': {'num_tokens': 1912,
  'num_articles': 6,
  'text': ['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pag

In [34]:
text = text_dict['dataset/raw_chunks/CAD220130244.pdf']['text']

In [35]:
import re
pattern = r"(\narticulo .*.*\s*\d+\d*).*:{0,1}\.*\s*"
pattern = "(?<=\n)(articulo .*.*\s*\d+\d*).*:{0,1}\.*\s*"
re.split(pattern=pattern, string=text[0])

['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\n',
 'articulo n° 1',
 'en caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efectos de

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

pattern = r"(?<=\n)articulo .*.*\s*\d+\d*.*:{0,1}\.*\s*"

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100 ,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len)

In [37]:
text[0]

'exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efect

In [38]:
r_splitter.split_text(text[0])

['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efec

In [39]:
from langchain.text_splitter import CharacterTextSplitter
from nltk import chunk

tiktoken_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    model_name=EMBEDDINGS_MODEL,
    chunk_size=1000,
    chunk_overlap=100,
    separator="\n",
)

In [40]:
texts = tiktoken_splitter.split_text(text[0])
texts

['exoneracion de pago de primas por fallecimiento del asegurado\ntitular\nincorporada al deposito de polizas bajo el codigo cad220130244\narticulo n° 1: cobertura.\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\nel valor de las primas a considerar para efectos de la ex

In [41]:
token_splits = tiktoken_splitter.split_documents(pages_dl)

Created a chunk of size 4286, which is longer than the specified 1000
Created a chunk of size 1682, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1198, which is longer than the specified 1000
Created a chunk of size 1273, which is longer than the specified 1000
Created a chunk of size 1271, which is longer than the specified 1000


In [42]:
encoder = tiktoken.encoding_for_model(MODEL)
def count_tokens(str):
    return len(encoder.encode(str))

In [43]:
for text_ in texts:
    print(count_tokens(text_))

928
948


In [44]:
count_tokens(text[0]) , sum([count_tokens(text_) for text_ in texts])

(1809, 1876)

In [45]:
splits = r_splitter.split_documents(pages_dl)

In [46]:

from langchain.vectorstores import Chroma

In [47]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [48]:
embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")

/home/andy/miniconda3/envs/chat_bot_31012/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
# db = Chroma.from_documents(splits,persist_directory="chroma", embedding=embeddings)

In [50]:
# db.persist()

In [51]:
from openai.api_resources import embedding


db = Chroma(persist_directory="chroma", embedding_function=embeddings)

In [52]:
# query it
query = "Que cubre la poliza POL120190177 PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS"
docs = db.similarity_search(query)

# print results
[doc.page_content for doc in docs]

['PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\n\nIncorporada al Depósito de Pólizas bajo el código POL120190177\n\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o el beneficiario.',
 'CONDICIONADO PARTICULAR: Documento que forma parte de la póliza, que especifica la fecha del inicio de vigencia, el deducible contratado, la suma asegurada, el tipo de plan, entre otra información de relevancia para el asegurado titular, y que además enumera a todas las personas cubiertas bajo la póliza de seguro.\n\nCONTRATANTE: La persona que firma la solicitud de incorporación para obtener cobertura, y que se obliga al pago de la prima.\n\nCONTRATO: El presente contrato

In [53]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [54]:
description = """ 
"The policy were the chunk is from, should in the following format `dataset/raw_chunks/{POLICY_NUMBER}.pdf` 
where POLICY_NUMBER is the policy number of the retrieved text. If the policy number is not stated in the document, DO NOT mix it with the 
policy name, such as PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO"""

metadata_field_info = [
    AttributeInfo(
        name="source",
        description=description,
        type="string",
    ),
]

In [55]:
import lark
document_content_description = "Lecture notes"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    db,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [56]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [57]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [80]:
from langchain.chat_models import ChatOpenAI

In [90]:
# Wrap our vectorstore
llm = ChatOpenAI(temperature=0, model_name=MODEL_16K, max_tokens=8_000)
compressor = LLMChainExtractor.from_llm(llm)

In [91]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [92]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [61]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = (
                "Combine the chat history and follow up question into "
                "a standalone question. Chat History: {chat_history}"
                "Follow up question: {question}"
            )
prompt = PromptTemplate.from_template(template)

question_generator_chain = LLMChain(llm=llm, prompt=prompt)

In [95]:
# Run chain
from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=compression_retriever,
    return_source_documents=False,
    memory=memory,
    chain_type="stuff"
)

In [96]:
question = "Que cubre el seguro todo riesgo de salud de la poliza CAD220130244.pdf"
result = qa_chain({"question": question})

/home/andy/miniconda3/envs/chat_bot_31012/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='Que cubre el seguro todo riesgo de salud' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='source', value='dataset/raw_chunks/CAD220130244.pdf') limit=None


In [97]:
result

{'question': 'Que cubre el seguro todo riesgo de salud de la poliza CAD220130244.pdf',
 'chat_history': [HumanMessage(content='Que cubre el seguro todo riesgo de salud de la poliza CAD220130244.pdf', additional_kwargs={}, example=False),
  AIMessage(content='El seguro todo riesgo de salud de la póliza CAD220130244.pdf cubre los gastos médicos y hospitalarios en caso de enfermedades o lesiones sufridas por el asegurado. Sin embargo, es importante revisar las Condiciones Particulares y las Condiciones Generales de la póliza para obtener información más detallada sobre la cobertura específica.', additional_kwargs={}, example=False)],
 'answer': 'El seguro todo riesgo de salud de la póliza CAD220130244.pdf cubre los gastos médicos y hospitalarios en caso de enfermedades o lesiones sufridas por el asegurado. Sin embargo, es importante revisar las Condiciones Particulares y las Condiciones Generales de la póliza para obtener información más detallada sobre la cobertura específica.'}

In [98]:
question = """
Dame mas informacion acerca de las Condiciones Particulares y las Condiciones Generales de la póliza para obtener información más detallada sobre la cobertura específica.

"""
result = qa_chain({"question": question})
result["answer"]

/home/andy/miniconda3/envs/chat_bot_31012/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='Condiciones Particulares Condiciones Generales póliza cobertura' filter=None limit=None


'No tengo acceso a las Condiciones Particulares y las Condiciones Generales de la póliza específica a la que te refieres. Te recomendaría que consultes directamente con la compañía aseguradora o revises la documentación que te han proporcionado para obtener información detallada sobre la cobertura y las condiciones específicas de tu póliza.'

In [99]:
result

{'question': '\nDame mas informacion acerca de las Condiciones Particulares y las Condiciones Generales de la póliza para obtener información más detallada sobre la cobertura específica.\n\n',
 'chat_history': [HumanMessage(content='Que cubre el seguro todo riesgo de salud de la poliza CAD220130244.pdf', additional_kwargs={}, example=False),
  AIMessage(content='El seguro todo riesgo de salud de la póliza CAD220130244.pdf cubre los gastos médicos y hospitalarios en caso de enfermedades o lesiones sufridas por el asegurado. Sin embargo, es importante revisar las Condiciones Particulares y las Condiciones Generales de la póliza para obtener información más detallada sobre la cobertura específica.', additional_kwargs={}, example=False),
  HumanMessage(content='\nDame mas informacion acerca de las Condiciones Particulares y las Condiciones Generales de la póliza para obtener información más detallada sobre la cobertura específica.\n\n', additional_kwargs={}, example=False),
  AIMessage(con